## Imports

In [ ]:
import pandas as pd
import numpy as np
import datetime
import requests

## Functions

### Get data

In [ ]:
import pandas as pd
import numpy as np
# import datetime # Removed to prevent overwriting datetime class with datetime module
import requests
import datetime # This import is not needed here as it is already imported below.
import requests
import zipfile
import io
import csv
import os
from datetime import datetime # This imports the datetime class from the datetime module.
from dateutil.relativedelta import relativedelta
from pytz import timezone


# Генерация списка месяцев
def generate_months(start_date, end_date):
    start_date = datetime.strptime(start_date, '%Y-%m')
    end_date = datetime.strptime(end_date, '%Y-%m')
    months = []

    while start_date <= end_date:
        months.append(start_date.strftime('%Y-%m'))
        start_date = start_date + relativedelta(months=1)

    return months


# Загрузка данных с Binance
def download_klines(symbol, interval, start_date, end_date):
    months = generate_months(start_date, end_date)
    if not os.path.exists('klines'):
        os.mkdir('klines')

    klines = {
        'Date': [], 'Open': [], 'High': [], 'Low': [], 'Close': [], 'Volume': []
    }

    for month in months:
        filename = f"{symbol}-{interval}-{month}.zip"
        file_path = f"klines/{filename}"

        if not os.path.exists(file_path) or os.path.getsize(file_path) == 0:
            url = f"https://data.binance.vision/data/futures/um/monthly/klines/{symbol}/{interval}/{filename}"
            try:
                r = requests.get(url, allow_redirects=True)
                with open(file_path, 'wb') as f:
                    f.write(r.content)
            except Exception as e:
                print(f"Ошибка загрузки файла {filename}: {e}. Пропускаем.")
                continue

        try:
            with zipfile.ZipFile(file_path, 'r') as zip_file:
                with zip_file.open(f"{symbol}-{interval}-{month}.csv", 'r') as csv_file:
                    csv_reader = csv.reader(io.TextIOWrapper(csv_file, 'utf-8'))
                    for row in csv_reader:
                        if row[0].isdigit():
                            # Use datetime.datetime.fromtimestamp to call the correct method.
                            klines['Date'].append(datetime.fromtimestamp(int(row[0]) / 1000, tz=timezone('UTC')))
                            klines['Open'].append(float(row[1]))
                            klines['High'].append(float(row[2]))
                            klines['Low'].append(float(row[3]))
                            klines['Close'].append(float(row[4]))
                            klines['Volume'].append(float(row[5]))
        except (zipfile.BadZipFile, KeyError):
            print(f"Ошибка: Файл {file_path} поврежден или не является ZIP. Пропускаем.")
            continue

    if not klines['Date']:
        raise ValueError(f"Нет доступных данных для {symbol} в диапазоне {start_date} - {end_date}")

    df = pd.DataFrame(klines)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)

    return df

data = download_klines('BTCUSDT', '5m', '2020-01', '2025-02')
data

,Open,High,Low,Close,Volume
Date,,,,,
2020-01-01 00:00:00+00:00,7189.43,7190.52,7172.94,7175.25,583.119
2020-01-01 00:05:00+00:00,7174.71,7186.69,7173.28,7186.69,163.860
2020-01-01 00:10:00+00:00,7186.83,7188.10,7175.70,7176.26,290.358
2020-01-01 00:15:00+00:00,7176.22,7179.41,7172.43,7173.68,189.617
2020-01-01 00:20:00+00:00,7173.50,7176.13,7170.69,7171.81,198.838
...,...,...,...,...,...
2025-02-28 23:35:00+00:00,84156.20,84216.00,84088.20,84215.90,143.780
2025-02-28 23:40:00+00:00,84216.00,84398.60,84215.90,84374.20,438.064
2025-02-28 23:45:00+00:00,84374.30,84513.10,84303.60,84468.50,697.042


In [ ]:
df = data.copy()

In [ ]:
df["max1"] = np.nan
df["min1"] = np.nan
df["max2"] = np.nan
df["max1_date"] = np.nan
df["min1_date"] = np.nan
df["max2_date"] = np.nan
signal = 0

last_max1 = None
last_min1 = None
last_max2 = None

date_last_max1 = None
date_last_min1 = None
date_last_max2 = None